<a href="https://colab.research.google.com/github/Yosingh1/Machine_Learning/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi #getting this error because GPU is not configure

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.27.2
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
be

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Successfully uninstalled accelerate-0.27.2
  Using cached transformers-4.38.1-py3-none-any.whl (8.5 MB)
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [5]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM

own translation system

language translation system

from the hugging face we are going to use open source translation model
data set also i wll tak


In [6]:
#finetuning of pre-trained model for Machine translation

In [7]:
#English to hindi translation

In [8]:
model="Helsinki-NLP/opus-mt-en-hi"

In [9]:
dataset=load_dataset("cfilt/iitb-english-hindi")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [11]:
# dataset["train"]["translation"]

In [12]:
dataset["train"]

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [13]:
dataset["test"]

Dataset({
    features: ['translation'],
    num_rows: 2507
})

In [14]:
dataset["validation"]

Dataset({
    features: ['translation'],
    num_rows: 520
})

In [15]:
dataset["train"][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [16]:
tokenizer=AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [17]:
tokenizer("Hi, This is yogendra") #converting sentenc in numeric representation

{'input_ids': [10595, 2, 239, 23, 34399, 10104, 30597, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenizer('अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें')

{'input_ids': [44, 4499, 2703, 314, 174, 44, 4499, 314, 3625, 2703, 1185, 428, 1321, 917, 3355, 44, 716, 917, 44, 2703, 24217, 549, 4228, 314, 130, 12604, 260, 44, 926, 1185, 1321, 260, 1321, 260, 1056, 44, 716, 260, 44, 800, 260, 7817, 44, 13087, 549, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
#target specific tokenization

with tokenizer.as_target_tokenizer():
  print(tokenizer('अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'))

{'input_ids': [63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [20]:
max_input_length = 128
max_target_length = 128

source_lang="en"
target_lang="hi"

def process_function(sentence):

  inputs=[ex[source_lang] for ex in sentence["translation"]]
  targets=[ex[target_lang] for ex in sentence["translation"]]

  model_inputs=tokenizer(inputs,max_length=max_input_length,truncation=True)

  with tokenizer.as_target_tokenizer():
    labels=tokenizer(targets,max_length=max_target_length,truncation=True)


  model_inputs["labels"]=labels["input_ids"]
  return model_inputs

In [21]:
tokenized_data=dataset.map(process_function, batched=True)

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [23]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [24]:
batch_size=32
learning_rate=0.001
weight_decay=0.01
num_train_epochs=1

In [25]:
data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [26]:
train_dataset=model.prepare_tf_dataset(
    tokenized_data["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn= data_collator

)

In [27]:
validation_dataset=model.prepare_tf_dataset(
    tokenized_data["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,

)

In [28]:
optimizer=AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)

In [29]:
model.compile(optimizer=optimizer)

In [30]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=num_train_epochs)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
78/78 [==============================] - 1637s 20s/step - loss: 4.5790 - val_loss: 4.8062


In [31]:
model.save_pretrained("tf_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


#model inferencing

In [32]:
input_text="Thousands of motorists have already taken the black boxes, some of which have GPS monitoring, for a test drive."

In [33]:
tokenized=tokenizer([input_text],return_tensors='np')

In [34]:
out=model.generate(**tokenized, max_length=128)

In [35]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True))

लाखों कारयानों ने ब्लैक बॉक्स को पहले से ही प्राप्त कर लिया है, जिसमें से कुछ के लिए एक जांच हो चुकी है।


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
